# Western US Long Term Fire Map (2012-2020)
This notebook uses data from the VIIRS satellites to plot detected fire events in the Western US over the period from 2012 to 2020. The data comes from NASA's S-NPP satellite via the FIRMS database. Data frome the newer NOAA-20 which came online in 2020 is excluded for more comperable visualizations over the years.

The data for each year is capped at the 255th day of the year which represents the latest cutoff for 2020 (September 11th). The heatplots represent cumulative detections for that year through that day.

In order to obtain the data needed, visit the FIRMS Archive Download tool https://firms.modaps.eosdis.nasa.gov/download/create.php. Use the bounding box tool to select the appropriate area and export the data as csv. Select both VIIRS satellites as both will make separate passes over the area giving the most coverage.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os
import shutil
from datetime import datetime
from pathlib import Path
from PIL import Image

In [2]:
def make_or_clear_path(path):
    if os.path.exists(path):
        shutil.rmtree(path)
        
    os.mkdir(path)
    
make_or_clear_path('long_range_out')

# Import Data
Convert the times to Pacific. Filter out data from the NOAA-20/JPSS-1 satellite designated as file name containing `J1`

In [122]:
def read_data(path):
    df = pd.read_csv(path, dtype={'acq_time': str})
    df['dt'] = pd.to_datetime(df['acq_date'] + ' ' + df['acq_time'])\
               .dt.tz_localize('UTC')\
               .dt.tz_convert('US/Pacific')\
               .dt.tz_localize(None)
    
    df['day'] = df['dt'].dt.dayofyear
    df['year'] = df['dt'].dt.year
    df = df[df['confidence'] != 'low']
    df['file'] = path
    df = df[df['file'].apply(lambda x: 'J1' not in x.name)]
    return df

# Change report paths as necessary
df = pd.concat([read_data(p) for p in Path('long_range_data').glob('*[JV]1*.csv')], axis=0)
df = df.sort_values(by='dt').set_index('dt')
df.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,day,year,file,brightness,bright_t31
dt,,,,,,,,,,,,,,,,,,,,
2012-01-20 01:43:00,34.094494,-117.531120,343.66,0.39,0.36,2012-01-20,0943,N,VIIRS,n,1,281.49,2.45,N,2.0,20,2012,long_range_data/fire_archive_V1_7088.csv,NaN,NaN
2012-01-20 01:43:00,34.243843,-118.380951,301.26,0.39,0.36,2012-01-20,0943,N,VIIRS,n,1,275.83,0.52,N,2.0,20,2012,long_range_data/fire_archive_V1_7088.csv,NaN,NaN
2012-01-20 01:43:00,34.324936,-118.527321,299.65,0.38,0.36,2012-01-20,0943,N,VIIRS,n,1,278.10,0.69,N,2.0,20,2012,long_range_data/fire_archive_V1_7088.csv,NaN,NaN
2012-01-20 01:43:00,34.035973,-118.104462,299.48,0.39,0.36,2012-01-20,0943,N,VIIRS,n,1,266.78,0.50,N,2.0,20,2012,long_range_data/fire_archive_V1_7088.csv,NaN,NaN
2012-01-20 01:43:00,34.319004,-118.511459,297.08,0.38,0.36,2012-01-20,0943,N,VIIRS,n,1,277.37,0.45,N,2.0,20,2012,long_range_data/fire_archive_V1_7088.csv,NaN,NaN


# Plot Fires
Plot fires using Plotly's density plot. The dark background requires a free Mapbox API token which is stored in a file named `token`. Outputs each fire to it's own folder.

We count the number of unique detections for each lon/lat point. The combination of range_cutoff, radius, image size, and zoom factor are related to each other and affect the perception of the heatmap. Choosing large values gives the impression the fires consume massive swaths of land while choosing small values reduces the fires to tiny pixels.

To add some objectivity, the map uses the maximum observed `fire radiative power` value through that point in the year. We set a threhhold using the 99.5 percentile to prevent some outlier values skewing the color shading.

In [162]:
with open('token', 'r') as f:
    token = f.read()
center = {'lat': 41.20, 'lon': -121.165}
cutoff = df['frp'].quantile(0.995)

In [167]:
def plot_fire(data, year, day, height, cutoff, center, token, output_path):
    df = data.loc[(data['year'] == year) & (data['day'] <= day)]\
        .reset_index()\
        .groupby(['year','latitude','longitude'])\
        .agg({'frp': 'max'})\
        .reset_index()

    fig = px.density_mapbox(df, lat="latitude", lon="longitude", z='frp',
                            center=center,
                            radius=2,
                            range_color=(0, cutoff),
                            color_continuous_scale=px.colors.sequential.YlOrRd,
                            zoom=5, height=height, width=int(height/2))
    fig.update_layout(
        mapbox_style="mapbox://styles/jbencina/ckezanwko03up19s65eopp1vp",
        mapbox_accesstoken=token,
        margin={'r':5,'l':5,'b':0},
        title={
            'text': f'{year}',
            'xanchor': 'center',
            'x': 0.5,
            'y': 0.98
        }
    )
    fig.update_layout({
        'title_font_color': '#FFFFFF',
        'title_font_family': 'Segoe UI',
        'title_font_size': 32,
        'plot_bgcolor': '#000000',
        'paper_bgcolor': '#000000',
        'coloraxis_showscale': False
    })
    save_path = os.path.join(output_path, f'{year}.jpg')
    fig.write_image(save_path)

for yr in range(2012, 2021):
    plot_fire(df, yr, 255, 1000, cutoff, center, token, 'long_range_out')
    print(f'Saved year {yr}')

Saved year 2012
Saved year 2013
Saved year 2014
Saved year 2015
Saved year 2016
Saved year 2017
Saved year 2018
Saved year 2019
Saved year 2020


# Make Single Image
Here we combine the 9 images into a single tiled image. We add a crop for the Mapbox copywrite since it repeats too many times but will manually add back when adding the finishing touches to this image in a post-processing step like Photoshop

In [168]:
def make_master_image(h, w, crop):
    r = 3
    c = 3

    img = Image.new('RGB', (w * 3, (h-crop) * 3))

    for i, yr in enumerate(range(2012, 2021)):
        s = Image.open(os.path.join('long_range_out', f'{yr}.jpg'))
        s = s.crop((0, 0, w, h-30))
        col = i % c
        row = i // r
        offset =  (col * w, row * (h-crop))
        img.paste(s, box=offset)
        img.save('2012-2020.jpg')

make_master_image(1000, 500, 30)

# Final Image - With External Edits
![map](processed.jpg)